<a href="https://colab.research.google.com/github/Jrodos12/Organizacion-de-Datos/blob/main/Analisis%20exploratorio%20de%20datos/Sparks_35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma? (⭐⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=539cf31e43c8520fb911b07f27ced1b19fd77e6504c32bb2fd0df0af33f3a395
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import datetime as dt

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext


In [ ]:
sqlContext = SQLContext(sc)
df_logs = sqlContext.read.csv("drive/MyDrive/logs.csv", header=True, inferSchema=True)
rdd_logs = df_logs.withColumn("fecha",to_timestamp(col("timestamp"))).select("title","fecha","action" ).rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
rdd_logs = rdd_logs.filter(lambda x: x.title != None and x.fecha != None )
# x.fecha != None: esta condicion la agrego porque hay algunos registros que donde tendria que ir la fecha esta colocado por error otro texto

In [ ]:
rdd_usuarios_creados = rdd_logs.filter( lambda x: x.action == 'create')
rdd_usuarios_bloqueados = rdd_logs.filter( lambda x: x.action == 'block')

In [ ]:
rdd_usuarios_creados.filter(lambda x: x.title =='Usuario:Namest 2003' ).take(1)

[Row(title='Usuario:Namest 2003', fecha=datetime.datetime(2021, 2, 12, 15, 50, 33), action='create')]

In [ ]:
rdd_usuarios_bloqueados.filter(lambda x: x.title =='Usuario:Namest 2003' ).take(3)

[Row(title='Usuario:Namest 2003', fecha=datetime.datetime(2019, 11, 16, 11, 5, 43), action='block'),
 Row(title='Usuario:Namest 2003', fecha=datetime.datetime(2019, 12, 12, 21, 3, 14), action='block'),
 Row(title='Usuario:Namest 2003', fecha=datetime.datetime(2021, 2, 12, 15, 46, 56), action='block')]

#como se ejemplifica mas arriba, existen usuarios que fueron bloqueados antes de ser creados (al menos por como se cargo en el logs, ya que estos resultados no tendrian sentidos porque la resta de cuando fue bloqueados - cuando fue creado seria negativa, decido filtrarlos y dejarlos fuera, dejo arriba codigo como justificacion de que dichos caso existen

In [ ]:
rdd_creados_bloqueados = rdd_usuarios_creados.join(rdd_usuarios_bloqueados).cache()

#Realizo las  operaciones por separado para que se entienda mejor que sucede en cada linea

In [ ]:
#transformo para cada usuario, el tiempo que tardo en ser bloqueado y lo paso a segundos
rdd_creados_bloqueados = rdd_creados_bloqueados.map(lambda x: (x[1][1] - x[1][0]).total_seconds() )

In [ ]:
#Como se explico mas arriba descarto los usuarios cuyo fecha de creacio con bloqueo no tienen sentido y generan resultados negativos
rdd_creados_bloqueados = rdd_creados_bloqueados.filter(lambda x: x > 0)

In [ ]:
#El resultado esta en segundos
rdd_creados_bloqueados.reduce(lambda x,y: x if x < y else y)

1.0